# **FOREWORD**

I am just starting out with this competition and saw this amazing post [here](https://www.kaggle.com/competitions/hull-tactical-market-prediction/discussion/608349) where the author highlights how to arrive at a score of 17++ on the public leaderboard! 

Let's try and implement this!

This is not going to generalize, so use with caution!!

In [ ]:
%%time 

from warnings import filterwarnings 
filterwarnings("ignore")
import os

import pandas as pd, numpy as np, polars as pl
from gc import collect 
from tqdm.notebook import tqdm
from scipy.optimize import minimize, Bounds

import kaggle_evaluation.default_inference_server

In [ ]:
%%time 

MIN_INVESTMENT = 0
MAX_INVESTMENT = 2


class ParticipantVisibleError(Exception):
    pass


def ScoreMetric(
    solution: pd.DataFrame, 
    submission: pd.DataFrame, 
    row_id_column_name: str
) -> float:
    """
    Calculates a custom evaluation metric (volatility-adjusted Sharpe ratio).

    This metric penalizes strategies that take on significantly more volatility
    than the underlying market.

    Returns:
        float: The calculated adjusted Sharpe ratio.
    """
    solution = solution
    solution['position'] = submission['prediction']

    if solution['position'].max() > MAX_INVESTMENT:
        raise ParticipantVisibleError(f'Position of {solution["position"].max()} exceeds maximum of {MAX_INVESTMENT}')
    if solution['position'].min() < MIN_INVESTMENT:
        raise ParticipantVisibleError(f'Position of {solution["position"].min()} below minimum of {MIN_INVESTMENT}')

    solution['strategy_returns'] = solution['risk_free_rate'] * (1 - solution['position']) + solution['position'] * solution['forward_returns']

    # Calculate strategy's Sharpe ratio
    strategy_excess_returns = solution['strategy_returns'] - solution['risk_free_rate']
    strategy_excess_cumulative = (1 + strategy_excess_returns).prod()
    strategy_mean_excess_return = (strategy_excess_cumulative) ** (1 / len(solution)) - 1
    strategy_std = solution['strategy_returns'].std()

    trading_days_per_yr = 252
    if strategy_std == 0:
        raise ZeroDivisionError
    sharpe = strategy_mean_excess_return / strategy_std * np.sqrt(trading_days_per_yr)
    strategy_volatility = float(strategy_std * np.sqrt(trading_days_per_yr) * 100)

    # Calculate market return and volatility
    market_excess_returns = solution['forward_returns'] - solution['risk_free_rate']
    market_excess_cumulative = (1 + market_excess_returns).prod()
    market_mean_excess_return = (market_excess_cumulative) ** (1 / len(solution)) - 1
    market_std = solution['forward_returns'].std()

    market_volatility = float(market_std * np.sqrt(trading_days_per_yr) * 100)

    # Calculate the volatility penalty
    excess_vol = max(0, strategy_volatility / market_volatility - 1.2) if market_volatility > 0 else 0
    vol_penalty = 1 + excess_vol

    # Calculate the return penalty
    return_gap = max(
        0,
        (market_mean_excess_return - strategy_mean_excess_return) * 100 * trading_days_per_yr,
    )
    return_penalty = 1 + (return_gap**2) / 100

    # Adjust the Sharpe ratio by the volatility and return penalty
    adjusted_sharpe = sharpe / (vol_penalty * return_penalty)
    return min(float(adjusted_sharpe), 1_000_000)



# **PUBLIC LB CHASER**

In [ ]:
%%time 

train = pd.read_csv(
    f"/kaggle/input/hull-tactical-market-prediction/train.csv", 
    index_col = "date_id"
)

def fun(x):
    "Source - https://www.kaggle.com/competitions/hull-tactical-market-prediction/discussion/608349"
    
    solution   =  train[-180:].copy()
    submission =  pd.DataFrame({'prediction': x.clip(0, 2)}, index=solution.index)
    return - ScoreMetric(solution, submission, '')

x0  = np.full(180, 0.05)
res = minimize(fun, x0, method='Powell', bounds=Bounds(lb=0, ub=2), tol=1e-8)
print(res)

opt_preds = res.x

# **INFERENCE**

In [ ]:
%%time 

i = 0
def predict(test: pl.DataFrame) -> float:
    
    global i, opt_preds
    
    pred = np.float64( opt_preds[i] )
    print(f"---> {pred:,.8f} | Iteration {i}")
    i = i + 1
    return pred

inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(('/kaggle/input/hull-tactical-market-prediction/',))


print()